<a href="https://colab.research.google.com/github/sravyagadam/ML_RecommenderSystems/blob/main/Book_Recommendation_Hybrid_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
books = pd.read_csv("/content/drive/MyDrive/Almabetter learning/Book Recommendation/Books.csv")
users = pd.read_csv("/content/drive/MyDrive/Almabetter learning/Book Recommendation/Users.csv")
ratings = pd.read_csv("/content/drive/MyDrive/Almabetter learning/Book Recommendation/Ratings.csv")

In [ ]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
ratings=ratings[ratings['Book-Rating']!=0]

In [ ]:
ratings.shape

(433671, 3)

In [ ]:
#pd.DataFrame(books['ISBN'].unique,columns=[['ISBN']])#['ISBN'].str.contains("X").sum()

In [ ]:
books.nunique(axis=0)
# ratings['ISBN'].unique

ISBN                   271360
Book-Title             242135
Book-Author            102023
Year-Of-Publication       202
Publisher               16807
Image-URL-S            271044
Image-URL-M            271044
Image-URL-L            271041
dtype: int64

In [ ]:
# Take only ISBN present in books df
ratings=ratings[ratings['ISBN'].isin(books['ISBN'])]
# Take users with more than 50 ratings
user_rating_counts=ratings.groupby('User-ID').size()
user_wthenough_rating_counts=user_rating_counts[user_rating_counts>50].reset_index()[['User-ID']]
df_with_selected_users=ratings.merge(user_wthenough_rating_counts,how='right',left_on='User-ID',right_on='User-ID')

In [ ]:
ratings.shape

(383842, 3)

In [ ]:
#df_with_selected_users=df_with_selected_users[df_with_selected_users['ISBN'].isin(books['ISBN'])]

In [ ]:
df_with_selected_users.head()

,User-ID,ISBN,Book-Rating
0,254,0060502320,7
1,254,0060934700,9
2,254,0060976977,7
3,254,0064471047,7
4,254,0066238501,5


In [ ]:
# user_ratn_book_merged=df_with_selected_users.merge(books,how='right',left_on='ISBN',right_on='ISBN')[['User-ID','ISBN','Book-Rating','Book-Title', 'Book-Author']]
# user_ratn_book_merged

In [ ]:
ratings.isna().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [ ]:
##Create a user rating matrix
user_rating_mtx_df = df_with_selected_users.pivot('User-ID','ISBN','Book-Rating').fillna(0)
print(f'Shape: {user_rating_mtx_df.shape}')

Shape: (1150, 85999)


In [ ]:
df_with_selected_users.nunique(axis=0)

User-ID         1150
ISBN           85999
Book-Rating       10
dtype: int64

In [ ]:
user_rating_mtx_df.head()

ISBN,0000913154,0001046438,000104687X,0001047213,0001047973,000104799X,0001048082,0001053736,0001053744,0001055607,0001056107,0001714600,0001821326,0001845039,0001935968,0001944711,0001945203,0001952803,0001953877,0001981625,0002000547,0002005018,0002005050,0002005557,0002006588,0002111314,0002115328,0002116286,0002118580,0002153572,0002154900,0002158973,0002163713,0002165368,0002176181,0002176432,0002179695,0002181924,0002184974,0002190915,...,9997409728,9997410440,9997411757,9997411870,9997412044,9997412958,9997507002,999750805X,9997508769,9997512952,9997519086,9997555635,9998914140,B00001U0CP,B00005B4LM,B00005N5J6,B00005TZWI,B00005U7YK,B000063VMD,B00006CRTE,B00006I4OX,B00006RGIO,B00007FYKW,B00008RWPV,B000092Q0A,B00009EF82,B00009NDAN,B0000C2W5U,B0000C7BNG,B0000DAPP1,B0000DYXID,B0000T6KHI,B0000T6KIM,B0000VZEJQ,B0000X8HIE,B00011SOXI,B00013AX9E,B0001GMSV2,B0001I1KOG,B000234N3A
User-ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_sim_matrx=cosine_similarity(user_rating_mtx_df)

In [ ]:
#pd.DataFrame(user_sim_matrx)

In [ ]:
## Based on similiar users
def book_recommender(user_rating_mtx, user_sim_matrx, user, k=20, top_n=10):
    # Get location of the actual movie in the User-Items matrix
    user_ix = user_rating_mtx.index.get_loc(user)
    # Use it to index the User similarity matrix
    user_similarities = user_sim_matrx[user_ix]
    # obtain the indices of the top k most similar users
    most_similar_users = user_rating_mtx.index[user_similarities.argpartition(-k)[-k:]]
    # Obtain the mean ratings of those users for all movies
    rec_books = user_rating_mtx.loc[most_similar_users].mean(0).sort_values(ascending=False)
    # Discard already seen movies
    books_rated_mtx = user_rating_mtx.loc[user].gt(0)
    books_rated = books_rated_mtx.index[books_rated_mtx].tolist()
    rec_books = rec_books.drop(books_rated).head(top_n)
    # return recommendations - top similar users rated movies
    return rec_books.index.to_frame().reset_index(drop=True).merge(books)

In [ ]:
book_recommender(user_rating_mtx_df, user_sim_matrx, user=276680, k=20, top_n=10)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0446672211,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,1998,Warner Books,http://images.amazon.com/images/P/0446672211.0...,http://images.amazon.com/images/P/0446672211.0...,http://images.amazon.com/images/P/0446672211.0...
1,0060934417,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,http://images.amazon.com/images/P/0060934417.0...,http://images.amazon.com/images/P/0060934417.0...
2,0671003755,She's Come Undone (Oprah's Book Club (Paperback)),Wally Lamb,1996,Washington Square Press,http://images.amazon.com/images/P/0671003755.0...,http://images.amazon.com/images/P/0671003755.0...,http://images.amazon.com/images/P/0671003755.0...
3,0345361792,A Prayer for Owen Meany,John Irving,1990,Ballantine Books,http://images.amazon.com/images/P/0345361792.0...,http://images.amazon.com/images/P/0345361792.0...,http://images.amazon.com/images/P/0345361792.0...
4,0312272057,Dry: A Memoir,Augusten Burroughs,2003,St. Martin's Press,http://images.amazon.com/images/P/0312272057.0...,http://images.amazon.com/images/P/0312272057.0...,http://images.amazon.com/images/P/0312272057.0...
5,0316601950,The Pilot's Wife : A Novel,Anita Shreve,1999,Back Bay Books,http://images.amazon.com/images/P/0316601950.0...,http://images.amazon.com/images/P/0316601950.0...,http://images.amazon.com/images/P/0316601950.0...
6,067976402X,Snow Falling on Cedars,David Guterson,1995,Vintage Books USA,http://images.amazon.com/images/P/067976402X.0...,http://images.amazon.com/images/P/067976402X.0...,http://images.amazon.com/images/P/067976402X.0...
7,0316776963,Me Talk Pretty One Day,David Sedaris,2001,Back Bay Books,http://images.amazon.com/images/P/0316776963.0...,http://images.amazon.com/images/P/0316776963.0...,http://images.amazon.com/images/P/0316776963.0...
8,0385498802,Bee Season: A Novel,Myla Goldberg,2001,Anchor Books/Doubleday,http://images.amazon.com/images/P/0385498802.0...,http://images.amazon.com/images/P/0385498802.0...,http://images.amazon.com/images/P/0385498802.0...
9,0743418174,Good in Bed,Jennifer Weiner,2002,Washington Square Press,http://images.amazon.com/images/P/0743418174.0...,http://images.amazon.com/images/P/0743418174.0...,http://images.amazon.com/images/P/0743418174.0...


In [ ]:
df_with_selected_users[df_with_selected_users['User-ID']==276680].merge(books)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276680,0020236107,7,FUNERAL MAKERS,Cathie Pelletier,1987,Scribner,http://images.amazon.com/images/P/0020236107.0...,http://images.amazon.com/images/P/0020236107.0...,http://images.amazon.com/images/P/0020236107.0...
1,276680,0060176784,7,Saying Grace: A Novel,Beth Gutcheon,1995,Harpercollins,http://images.amazon.com/images/P/0060176784.0...,http://images.amazon.com/images/P/0060176784.0...,http://images.amazon.com/images/P/0060176784.0...
2,276680,0060283130,7,The Reptile Room (A Series of Unfortunate Even...,Lemony Snicket,1999,HarperCollins,http://images.amazon.com/images/P/0060283130.0...,http://images.amazon.com/images/P/0060283130.0...,http://images.amazon.com/images/P/0060283130.0...
3,276680,0060925493,7,Feather Crowns,Bobbie Ann Mason,1994,Perennial,http://images.amazon.com/images/P/0060925493.0...,http://images.amazon.com/images/P/0060925493.0...,http://images.amazon.com/images/P/0060925493.0...
4,276680,0060933429,10,A Ship Made of Paper : A Novel,Scott Spencer,2004,Ecco,http://images.amazon.com/images/P/0060933429.0...,http://images.amazon.com/images/P/0060933429.0...,http://images.amazon.com/images/P/0060933429.0...
...,...,...,...,...,...,...,...,...,...,...
85,276680,1573222038,10,Fingersmith,Sarah Waters,2002,Riverhead Hardcover,http://images.amazon.com/images/P/1573222038.0...,http://images.amazon.com/images/P/1573222038.0...,http://images.amazon.com/images/P/1573222038.0...
86,276680,1573229083,7,My Dream of You,Nuala O'Faolain,2002,Riverhead Books,http://images.amazon.com/images/P/1573229083.0...,http://images.amazon.com/images/P/1573229083.0...,http://images.amazon.com/images/P/1573229083.0...
87,276680,1888173408,7,Fairy Tale (Common Reader Editions),Alice Thomas Ellis,2001,Akadine Press,http://images.amazon.com/images/P/1888173408.0...,http://images.amazon.com/images/P/1888173408.0...,http://images.amazon.com/images/P/1888173408.0...
88,276680,1888173564,8,Parnassus on Wheels (Common Reader Editions),Christopher Morley,2000,Joiner/Oriel Inc,http://images.amazon.com/images/P/1888173564.0...,http://images.amazon.com/images/P/1888173564.0...,http://images.amazon.com/images/P/1888173564.0...


In [ ]:
user_rating_mtx_df = df_with_selected_users.pivot('User-ID','ISBN','Book-Rating').fillna(0)

In [ ]:
interactions_full_df=df_with_selected_users.copy()

In [ ]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['User-ID'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 121539
# interactions on Test set: 30385


In [ ]:
user_rating_mtx_df = interactions_train_df.pivot('User-ID','ISBN','Book-Rating').fillna(0)

In [ ]:
user_rating_mtx=user_rating_mtx_df.values

In [ ]:
# The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(user_rating_mtx, k = NUMBER_OF_FACTORS_MF)

In [ ]:
user_rating_mtx.shape

(1150, 72696)

In [ ]:
U.shape

(1150, 15)

In [ ]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [ ]:
Vt.shape

(15, 72696)

In [ ]:
np.dot(U, sigma).shape

(1150, 15)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
all_user_predicted_ratings

array([[-0.00833441,  0.0042161 ,  0.00281073, ...,  0.00455455,
         0.00203003,  0.03211756],
       [-0.01459855, -0.00910411, -0.0060694 , ...,  0.00396314,
         0.00188417,  0.01531663],
       [-0.00432312,  0.04594478,  0.03062985, ...,  0.00112231,
         0.0005412 ,  0.00565481],
       ...,
       [-0.00147262, -0.0017175 , -0.001145  , ...,  0.00094048,
         0.00040899,  0.00466164],
       [-0.0020797 , -0.00145659, -0.00097106, ...,  0.00051506,
         0.0002531 ,  0.00240062],
       [ 0.01453206,  0.00530234,  0.00353489, ...,  0.00096708,
         0.00040501,  0.00687006]])

In [ ]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = user_rating_mtx_df.columns, index=user_rating_mtx_df.index).transpose()
cf_preds_df.head()

User-ID,254,638,1424,1733,1903,2033,2110,2276,2766,2891,2977,3363,3757,3827,4017,4385,5209,5582,6073,6242,6251,6323,6543,6563,6575,7125,7158,7286,7346,7915,8066,8067,8245,8253,8454,8681,8890,8936,8971,9613,...,267033,267635,267830,268030,268032,268110,268300,268657,268932,269157,269321,269557,269566,269728,269890,270352,270605,270629,270713,270820,270838,271195,271448,271622,271705,272225,272482,273113,273718,273979,274004,274061,274111,274301,275970,276680,277427,278137,278188,278418
ISBN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0000913154,-0.008334,-0.014599,-0.004323,-0.007654,-0.000011,0.074123,-0.002808,0.010478,0.011418,0.012983,-0.002834,-0.005112,-8.224628e-05,0.031630,-0.004777,-0.006039,0.000001,0.009274,0.001324,-0.008675,0.026878,-0.001386,0.014784,-0.016530,0.030382,-0.008183,-0.008285,-0.003067,0.035329,-0.013559,-0.006735,-0.018495,0.015227,-0.005079,-0.008761,-0.010119,-0.000062,0.003892,0.020777,0.033200,...,0.006965,0.012930,-0.002535,-0.003113,0.017109,0.021879,0.008802,-0.007364,0.011042,-0.002950,-0.004923,-0.001356,-0.011687,-0.000527,0.009604,-0.003905,-0.011206,-0.000192,-0.014270,-0.015612,-0.004230,0.030446,-0.029523,-0.001582,0.022037,0.007900,0.000063,-0.015579,-0.003546,-0.008477,-0.003636,-0.016097,-0.000541,-0.001463,0.016051,-0.010431,0.054282,-0.001473,-0.002080,0.014532
0001046438,0.004216,-0.009104,0.045945,-0.004812,-0.001550,0.005143,-0.003941,-0.007498,0.007909,-0.003860,0.007775,-0.004345,-6.693591e-07,-0.002457,0.054327,-0.002740,-0.000706,-0.002828,0.029066,0.035875,0.017050,-0.001856,-0.012359,-0.011711,0.045786,-0.006594,-0.005865,-0.001180,0.010407,-0.001417,0.002463,-0.016065,0.021514,0.010851,0.016794,0.007811,0.000118,-0.002905,-0.000219,-0.001828,...,0.014613,-0.003642,-0.004220,0.021362,0.024696,-0.008458,-0.003767,0.008469,-0.005108,-0.003432,-0.003402,-0.001849,-0.010770,0.006337,-0.005853,-0.003305,0.005030,-0.000075,0.038331,-0.010660,-0.003055,-0.006702,-0.015434,0.013733,0.057356,-0.003042,0.000451,-0.009893,0.031133,0.006355,0.011690,-0.033514,0.000271,0.050203,-0.006449,0.008256,-0.008719,-0.001717,-0.001457,0.005302
000104687X,0.002811,-0.006069,0.030630,-0.003208,-0.001033,0.003429,-0.002627,-0.004999,0.005273,-0.002574,0.005183,-0.002897,-4.462394e-07,-0.001638,0.036218,-0.001827,-0.000471,-0.001885,0.019377,0.023917,0.011367,-0.001238,-0.008240,-0.007807,0.030524,-0.004396,-0.003910,-0.000787,0.006938,-0.000945,0.001642,-0.010710,0.014343,0.007234,0.011196,0.005207,0.000079,-0.001936,-0.000146,-0.001219,...,0.009742,-0.002428,-0.002813,0.014241,0.016464,-0.005639,-0.002511,0.005646,-0.003405,-0.002288,-0.002268,-0.001232,-0.007180,0.004225,-0.003902,-0.002203,0.003353,-0.000050,0.025554,-0.007107,-0.002037,-0.004468,-0.010289,0.009155,0.038237,-0.002028,0.000301,-0.006596,0.020756,0.004237,0.007793,-0.022343,0.000181,0.033469,-0.004299,0.005504,-0.005812,-0.001145,-0.000971,0.003535
000104799X,0.005732,0.005832,0.001439,0.002047,0.000989,0.004659,0.002047,0.003012,0.004584,0.003308,0.001751,0.002465,2.915613e-05,0.002040,0.012714,0.001018,0.000351,0.005888,0.002620,0.006347,0.011783,0.001485,0.007962,0.007579,0.021079,0.003868,0.003132,0.001509,0.021028,0.006349,0.004492,0.007137,0.007222,0.010331,0.006317,0.004775,-0.000054,0.001653,0.000232,0.000680,...,0.002603,0.002084,0.002404,0.002365,0.005203,0.007805,0.008130,0.004607,0.002977,0.001885,0.000927,0.000823,0.006745,0.000979,0.003273,0.001522,0.005423,0.000271,0.013232,0.006536,0.001861,0.004648,0.015075,0.001649,0.008372,0.002361,0.000078,0.003091,0.001620,0.003628,0.005010,0.011577,0.000361,0.010899,0.003991,0.004741,0.005581,0.001294,0.000756,0.001144
0001048082,0.009184,-0.002209,0.042177,-0.002611,-0.000606,0.009381,-0.001863,-0.004136,0.010834,-0.000684,0.008554,-0.001186,2.221658e-05,-0.000584,0.058959,-0.001777,-0.000349,0.002462,0.027811,0.037071,0.026022,-0.000577,-0.004889,-0.003825,0.058377,-0.002518,-0.001814,0.000301,0.024744,0.004174,0.005721,-0.009219,0.0254

In [ ]:
cf_preds_df.shape

(72696, 1150)

In [ ]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating content that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['ISBN'].isin(items_to_ignore)].sort_values('recStrength', ascending = False).head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'ISBN', 
                                                          right_on = 'ISBN')[['User-ID','ISBN','Book-Title', 'Book-Author']]


        return recommendations_df
    
cf_recommender_model = CFRecommender(cf_preds_df, books)

In [ ]:
cf_recommender_model

In [ ]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('User-ID')
interactions_train_indexed_df = interactions_train_df.set_index('User-ID')
interactions_test_indexed_df = interactions_test_df.set_index('User-ID')

### The Top-N accuracy metric choosen was **Recall@N** which evaluates whether the interacted item is among the top N items (hit) in the ranked list of 101 recommendations for a user.

In [ ]:
def get_items_interacted(user_id, interactions_df):
    interacted_items = interactions_df.loc[user_id]['ISBN']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [ ]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:

    # Function for getting the set of items which a user has not interacted with
    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(interactions_full_df['ISBN'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    # Function to verify whether a particular item_id was present in the set of top N recommended items
    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index
    
    # Function to evaluate the performance of model for each user
    def evaluate_model_for_user(self, model, person_id):
        
        # Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        
        if type(interacted_values_testset['ISBN']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['ISBN'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['ISBN'])])
            
        interacted_items_count_testset = len(person_interacted_items_testset) 

        # Getting a ranked recommendation list from the model for a given user
        person_recs_df = model.recommend_items(person_id, items_to_ignore=get_items_interacted(person_id, interactions_train_indexed_df),topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        
        # For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            
            # Getting a random sample of 100 items the user has not interacted with
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS) #, seed=item_id%(2**32)

            # Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            # Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['ISBN'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['ISBN'].values
            
            # Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        # Recall is the rate of the interacted items that are ranked among the Top-N recommended items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    
    # Function to evaluate the performance of model at overall level
    def evaluate_model(self, model):
        
        people_metrics = []
        
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):    
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
            
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics).sort_values('interacted_count', ascending=False)
        No
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()

In [ ]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)

print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...
1149 users processed

Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@5': 0.2249136086884976, 'recall@10': 0.29353299325324994}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
32,156,254,1389,0.112311,0.182865,11676
26,180,228,1138,0.158172,0.200351,98391
33,20,22,380,0.052632,0.057895,189835
43,84,110,369,0.227642,0.298103,153662
11,25,30,236,0.105932,0.127119,23902
177,28,34,204,0.137255,0.166667,235105
180,11,18,203,0.054187,0.088670,76499
89,23,33,193,0.119171,0.170984,171118
6,78,80,192,0.406250,0.416667,16795
85,20,30,188,0.106383,0.159574,248718


In [ ]:
########### create an item item sim matrix

In [ ]:
user_book_ratng_mtx=df_with_selected_users.merge(books,how='inner',left_on='ISBN',right_on='ISBN')[['User-ID','ISBN','Book-Title', 'Book-Author','Book-Rating']]
user_book_ratng_mtx['Book-Author-Title']=user_book_ratng_mtx['Book-Title'].astype(str) +" "+ user_book_ratng_mtx['Book-Author']
user_book_ratng_mtx.drop(columns=['Book-Title', 'Book-Author'],axis=1,inplace=True)

In [ ]:
user_book_ratng_mtx.shape

(151924, 4)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Ignoring stopwords (words with no semantics) from English and Portuguese (as we have a corpus with mixed languages)

stopwords_list = stopwords.words('english')

#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(2,4),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

item_ids = user_book_ratng_mtx['ISBN'].tolist()
tfidf_matrix = vectorizer.fit_transform(user_book_ratng_mtx['Book-Author-Title'])
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix

<151924x21 sparse matrix of type '<class 'numpy.float64'>'
	with 15833 stored elements in Compressed Sparse Row format>

In [ ]:
#tfidf_matrix_dense=tfidf_matrix.todense()

In [ ]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['ISBN'])
    
    user_item_strengths = np.array(interactions_person_df['Book-Rating']).reshape(-1,1)
    
    # Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = interactions_full_df[interactions_full_df['ISBN'].isin(user_book_ratng_mtx['ISBN'])].set_index('User-ID')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [ ]:
user_profiles = build_users_profiles()
len(user_profiles)

1150

In [ ]:
#Look at one particular user profile
user_profile = user_profiles[189835]
print(user_profile.shape)

pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles[189835].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

(1, 21)


,token,relevance
0,stephen king,0.999602
1,harry potter,0.022713
2,book rowling,0.009729
3,john grisham,0.008400
4,nora roberts,0.008400
5,danielle steel,0.006720
6,anne mccaffrey,0.000000
7,anne rice,0.000000
8,book club,0.000000
9,dean koontz,0.000000


In [ ]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        
        # Compute the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        
        # Get the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        
        # Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['ISBN', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'ISBN', 
                                                          right_on = 'ISBN')[['User-ID','ISBN','Book-Title', 'Book-Author']]


        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(books)

# Evaluation

In [ ]:
def get_items_interacted(user_id, interactions_df):
    interacted_items = interactions_df.loc[user_id]['ISBN']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [ ]:
model_evaluator = ModelEvaluator()    

In [ ]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)


print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
1149 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.03853875267401678, 'recall@10': 0.03906532828698371}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
32,12,12,1389,0.008639,0.008639,11676
26,1,1,1138,0.000879,0.000879,98391
33,11,11,380,0.028947,0.028947,189835
43,8,8,369,0.021680,0.021680,153662
11,0,0,236,0.000000,0.000000,23902
177,2,2,204,0.009804,0.009804,235105
180,0,0,203,0.000000,0.000000,76499
89,1,1,193,0.005181,0.005181,171118
6,4,4,192,0.020833,0.020833,16795
85,6,6,188,0.031915,0.031915,248718
